# Submission 3: Ensemble Model

In [ ]:
import pandas as pd

print('--- Running Submission 3: Ensemble Model ---')

# Simulate creating a final submission file
submission_df = pd.DataFrame({
    'battle_id': range(10),
    'prediction': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0] # Yet another prediction pattern
})

submission_df.to_csv('submission_3.csv', index=False)

print('✅ Submission 3 complete. `submission_3.csv` created.')